# Connect driver

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Install lib

In [2]:
!pip install evaluate
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 876.3 kB/s eta 0:00:000:00:010m


In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d623a49ed82d33570c439371f4184db9f0bfbdd05c150432d30e72714b560095
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [4]:
!pip install accelerate

In [5]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 2.2 MB/s eta 0:00:0000:0100:01m


In [6]:
import os
os.environ["WANDB_DISABLED"] = "true"

# Import Lib

In [7]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM

2024-05-21 06:27:22.185427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 06:27:22.185535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 06:27:22.438529: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Init Model

In [8]:
MODEL_NAME = "VietAI/envit5-base"
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

spiece.model:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


pytorch_model.bin:   0%|          | 0.00/1.10G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Build dataset

In [9]:
dataset = load_dataset("ncduy/mt-en-vi")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 2884451
    })
    validation: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 11316
    })
    test: Dataset({
        features: ['en', 'vi', 'source'],
        num_rows: 11225
    })
})

In [10]:
dataset_train = dataset["train"].select(range(80000))
dataset_valid = dataset["validation"]
dataset_test = dataset["test"]

In [11]:
def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   inputs = [doc for doc in examples["en"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)

   labels = tokenizer(text_target=examples["vi"],
                      max_length=128,
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [12]:
# Map the preprocessing function across our dataset
tokenized_dataset_train = dataset_train.map(preprocess_function, batched=True)
tokenized_dataset_valid = dataset_valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/80000 [00:00<?, ? examples/s]

Map:   0%|          | 0/11316 [00:00<?, ? examples/s]

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/kaggle/working/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,1.767500,1.411794
2,1.586800,1.375269


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead u

TrainOutput(global_step=10000, training_loss=1.7874607666015625, metrics={'train_runtime': 6315.5455, 'train_samples_per_second': 25.334, 'train_steps_per_second': 1.583, 'total_flos': 1.2433768590606336e+16, 'train_loss': 1.7874607666015625, 'epoch': 2.0})

In [49]:
model_path = "/kaggle/working/checkpoint-10000"

In [66]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

def translation(sentence):
    input_ids = tokenizer.encode(sentence, return_tensors="pt").to(device)
    translated_text = model.generate(input_ids.to(device))
    translated_text_decoded = tokenizer.decode(translated_text[0], skip_special_tokens=True)
    return translated_text_decoded

In [67]:
translation("Never say everything that you know. And never believe everything that you hear.")

'Đừng nói những gì bạn biết và đừng tin những gì bạn nghe.'

In [68]:
import pandas as pd

In [69]:
test_df = pd.DataFrame(dataset_test)

In [70]:
test_df = test_df.drop(test_df.columns[2], axis=1)
test_df.shape

(11225, 2)

In [71]:
test_df

,en,vi
0,And what I think the world needs now is more c...,Và tôi nghĩ điều thế giới đang cần bây giờ là ...
1,"The group is named after Bangkok, the capital ...","Nhóm được đặt theo tên của Bangkok, thủ đô của..."
2,It is surrounded by rivers (Simpson and Coyhai...,Nó được bao quanh bởi các con sông (Simpson và...
3,"Four years, and you never talked to her.","Bốn năm, và cậu chưa nói gì với cô ấy."
4,The man that's giving the test has serious dou...,Người cung cấp bài kiểm tra đó đang nghi ngờ v...
...,...,...
11220,It's important.,Đây là điều quan trọng.
11221,I looked at him and I saw myself.,"Mình nhìn vào nó, và nhìn thấy chính con người..."
11222,In India it is distributed mainly on the plain...,"Tại Ấn Độ, nó phân bố chủ yếu trên các vùng đồ..."
11223,He moved to MIO Biwako Shiga in 2015.,Anh chuyển đến MIO Biwako Shiga năm 2015.


In [ ]:
test_df['vi_candidate'] = test_df['en'].apply(lambda x: translation(x))

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
test_df

In [ ]:
test_df['vi_length'] = test_df['vi'].apply(lambda x: len(x.split()))
test_df.head()

In [ ]:
test_df.to_csv('/kaggle/working/Output_with_T5.csv', index=False)